In [29]:
from datasets import load_dataset
import pickle

In [30]:
dataset_availability = {}
with open("../pkl/available_kb_datasets.pkl","rb") as pkl: dataset_availability["kb_available"] = pickle.load(pkl)
with open("../pkl/unavailable_kb_datasets.pkl","rb") as pkl: dataset_availability["kb_unavailable"] = pickle.load(pkl)
with open("../pkl/available_qa_datasets.pkl","rb") as pkl: dataset_availability["qa_available"] = pickle.load(pkl)
with open("../pkl/unavailable_qa_datasets.pkl","rb") as pkl: dataset_availability["qa_unavailable"] = pickle.load(pkl)
with open("../pkl/available_t2t_datasets.pkl","rb") as pkl: dataset_availability["t2t_available"] = pickle.load(pkl)
with open("../pkl/unavailable_t2t_datasets.pkl","rb") as pkl: dataset_availability["t2t_unavailable"] = pickle.load(pkl)
with open("../pkl/available_text_datasets.pkl","rb") as pkl: dataset_availability["text_available"] = pickle.load(pkl)
with open("../pkl/unavailable_text_datasets.pkl","rb") as pkl: dataset_availability["text_unavailable"] = pickle.load(pkl)

In [31]:
def get_entities(df):    
    try:    
        entities = df[df["entities"].apply(len) != 0]["entities"].tolist()
        entities = [item for sublist in entities for item in sublist]
        new_entities = {}
        for entity in entities: new_entities[entity["id"]] = f"{entity['type']}"
        for entity in set([new_entities[entity] for entity in new_entities.keys()]): print(entity)
        print("")
        return new_entities
    except:
        return []
    
def get_relations(df,entities):
    try:
        relations = df[df["relations"].apply(len) != 0]["relations"].tolist()
        relations = [item for sublist in relations for item in sublist]
        for relation in set([f"{entities[relation['arg1_id']]} - {relation['type'].upper()} - {entities[relation['arg2_id']]}" for relation in relations]):
            print(relation)
        
    except:
        pass

def select_largest_split(dataset_dict):
    max_split = None
    max_rows = 0
    max_dataset = None

    for split, dataset in dataset_dict.items():
        if dataset.num_rows > max_rows:
            max_split = split
            max_rows = dataset.num_rows
            max_dataset = dataset
    
    return max_split, max_rows, max_dataset

In [32]:
for elem in dataset_availability["kb_available"]:
    dataset = load_dataset(f"bigbio/{elem[0]}",name=elem[1],trust_remote_code=True)
    split, rows, df = select_largest_split(dataset)
    df = df.to_pandas()
    print(f"DATASET\n{elem} - {split} - {rows}\n\nENTITIES:")
    entities = get_entities(df)
    print(f"RELATIONS:")
    get_relations(df,entities)
    print("\n--------------------------------------------------------------------\n")

DATASET
('an_em', 'an_em_bigbio_kb') - train - 250

ENTITIES:
Organism_subdivision
Tissue
Organism_substance
Developing_anatomical_structure
Pathological_formation
Immaterial_anatomical_entity
Cellular_component
Multi-tissue_structure
Organ
Anatomical_system
Cell

RELATIONS:
Multi-tissue_structure - FRAG - Multi-tissue_structure
Organism_substance - PART-OF - Organism_substance
Cell - FRAG - Cell
Pathological_formation - PART-OF - Organ
Organism_substance - PART-OF - Cell
Cell - PART-OF - Multi-tissue_structure
Cellular_component - FRAG - Cellular_component
Tissue - PART-OF - Tissue
Pathological_formation - PART-OF - Pathological_formation
Organism_substance - FRAG - Organism_substance
Cell - PART-OF - Pathological_formation
Tissue - FRAG - Tissue
Pathological_formation - FRAG - Pathological_formation
Organ - FRAG - Organ
Multi-tissue_structure - PART-OF - Multi-tissue_structure
Cell - PART-OF - Cell
Multi-tissue_structure - PART-OF - Organ
Multi-tissue_structure - PART-OF - Developing